In [11]:
from parametros import densidad_pulpa, ge, tonelaje, dimensiones_tanques, capacidad_tanques
from data import integracion_data
from sim_tanques import Tanques
from cosecha_tanques import Cosecha 
from trazabilidad import Trazabilidad
import pandas as pd
from itertools import combinations
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

minutos = 5
intervalo = 5
tiempo_total = 24*60*25

root_mineral = '/Users/michaelcueto/Documents/proyectos/trazavilidad-VetaDorada/version1.0/database/mineral.xlsx'
root_blending = '/Users/michaelcueto/Documents/proyectos/trazavilidad-VetaDorada/version1.0/database/blendings.xlsx'
root_recuperacion = '/Users/michaelcueto/Documents/proyectos/trazavilidad-VetaDorada/version1.0/database/recuperacion.xlsx'
root_fecha = '/Users/michaelcueto/Documents/proyectos/trazavilidad-VetaDorada/version1.0/database/fecha_blending.xlsx'


df_materiales = integracion_data(root_mineral, root_blending, root_recuperacion, root_fecha, minutos).mill_to_lix(deltatime=45)
df_info_blendings = integracion_data(root_mineral, root_blending, root_recuperacion, root_fecha, minutos).info_blending(rec_estandar=0.90)
df_tk1, df_tk2, df_tk3, df_tk4, df_tk5, df_tk6, df_tk7, df_tk8, df_tk9, df_tk10, df_tk11 = Tanques(df_materiales, capacidad_tanques, dimensiones_tanques, tonelaje, densidad_pulpa, ge, intervalo, tiempo_total).simular_tanques()

In [15]:
import pandas as pd
import numpy as np

# Definir funciones

def aplicar_cosecha(df, fechas_cosecha):
    """
    Borra los valores (excepto "time") dentro del intervalo de cosecha.
    :param df: DataFrame con datos del tanque.
    :param fechas_cosecha: Lista de tuplas (inicio, fin) con fechas de cosecha.
    """
    for inicio, fin in fechas_cosecha:
        df.loc[(df["time"] >= inicio) & (df["time"] <= fin), df.columns.difference(["time"])] = None
    return df

def aplicar_transferencia(df_i, df_j, fechas_transferencia):
    """
    Borra valores en tanque j dentro del intervalo y llena con datos de tanque i.
    :param df_i: DataFrame del tanque de origen.
    :param df_j: DataFrame del tanque destino.
    :param fechas_transferencia: Lista de tuplas (inicio, fin) con fechas de transferencia.
    """
    for inicio, fin in fechas_transferencia:
        mask = (df_j["time"] >= inicio) & (df_j["time"] <= fin)
        df_j.loc[mask, df_j.columns.difference(["time"])] = None  # Borrar datos del tanque j
        df_j.update(df_i[df_i["time"].between(inicio, fin)])  # Llenar con datos del tanque i
    return df_j

# Crear DataFrames con datos simulados
fechas_nuevas = pd.date_range("2025-01-01 00:00:00", "2025-01-10 23:00:00", freq="60T")

df_t1 = pd.DataFrame({
    "time": fechas_nuevas,
    "pH": np.round(np.random.uniform(6.5, 8.5, len(fechas_nuevas)), 2),
    "DO": np.round(np.random.uniform(5.0, 10.0, len(fechas_nuevas)), 2),
    "Temp": np.round(np.random.uniform(20.0, 25.0, len(fechas_nuevas)), 2)
})

df_t2 = pd.DataFrame({
    "time": fechas_nuevas,
    "pH": np.round(np.random.uniform(6.5, 8.5, len(fechas_nuevas)), 2),
    "DO": np.round(np.random.uniform(5.0, 10.0, len(fechas_nuevas)), 2),
    "Temp": np.round(np.random.uniform(20.0, 25.0, len(fechas_nuevas)), 2)
})

df_t3 = pd.DataFrame({
    "time": fechas_nuevas,
    "pH": np.round(np.random.uniform(6.5, 8.5, len(fechas_nuevas)), 2),
    "DO": np.round(np.random.uniform(5.0, 10.0, len(fechas_nuevas)), 2),
    "Temp": np.round(np.random.uniform(20.0, 25.0, len(fechas_nuevas)), 2)
})

# Definir periodos de cosecha y transferencia (listas de tuplas)
fechas_cosecha = [(pd.Timestamp("2025-01-05 10:00:00"), pd.Timestamp("2025-01-05 13:00:00"))]
fechas_transferencia = [(pd.Timestamp("2025-01-06 10:00:00"), pd.Timestamp("2025-01-06 12:00:00"))]

# Aplicar las funciones
df_t1_mod = aplicar_cosecha(df_t1.copy(), fechas_cosecha)
df_t2_mod = aplicar_transferencia(df_t3.copy(), df_t2.copy(), fechas_transferencia)

# Mostrar resultados
df_t1_mod

,time,pH,DO,Temp
0,2025-01-01 00:00:00,8.04,7.48,20.88
1,2025-01-01 01:00:00,6.60,7.82,24.53
2,2025-01-01 02:00:00,7.53,8.91,21.16
3,2025-01-01 03:00:00,6.94,5.56,23.87
4,2025-01-01 04:00:00,6.66,9.41,20.22
...,...,...,...,...
235,2025-01-10 19:00:00,7.34,7.84,22.94
236,2025-01-10 20:00:00,7.67,8.41,23.78
237,2025-01-10 21:00:00,8.36,6.26,21.02
238,2025-01-10 22:00:00,7.36,5.06,24.47
